In [ ]:
"""
Solution 1:
Target one log and count whether next logs are within 1000ms
O(n**2)
"""
from collections import defaultdict

# Debug
def ms_to_hmsms(time):
    tc = {
        "s": 1000,
        "m": 1000 * 60,
        "h": 1000 * 60 * 60
    }
    h, time = divmod(time, tc["h"])
    m, time = divmod(time, tc["m"])
    s, ms = divmod(time, tc["s"])
    print(h, m, s, ms)


def parsing(line):
    # time conversion to ms
    tc = {
        "s": 1000,
        "m": 1000 * 60,
        "h": 1000 * 60 * 60
    }

    # 로그 문자열마다 요청에 대한 응답완료시간 S와 처리시간 T가 공백으로 구분
    # S는 작년 추석인 2016년 9월 15일만 포함하여 고정 길이 2016-09-15 hh:mm:ss.sss 형식
    _, S, T = line.split()
    h, m, s = S.split(":")
    s, ms = s.split(".")

    # 처리시간 T는 최대 소수점 셋째 자리까지 기록하며 뒤에는 초 단위를 의미하는 s
    T = T.rstrip("s")
    T_s, T_ms = 0, 0
    if "." in T:
        T_s, T_ms = T.split(".")
    else:
        T_s = T

    # Convert time into ms
    h, m, s, ms, T_s, T_ms = tuple(map(int, [h, m, s, ms, T_s, T_ms]))
    end_response = ms + (s * tc["s"]) + (m * tc["m"]) + (h * tc["h"])

    # 처리시간은 시작시간과 끝시간을 포함
    T = T_ms + (T_s * tc["s"])
    start_response = end_response - T + 1

    return (start_response, end_response)


def solution(lines):
    answer = 1 # lines 배열은 N(1 ≦ N ≦ 2,000)개

    checkpoint = list()
    # lines 배열은 N(1 ≦ N ≦ 2,000)개의 로그 문자열
    # lines 배열은 응답완료시간 S를 기준으로 오름차순 정렬되어 있다
    LEN = len(lines)
    for i in range(LEN - 1):
        line = lines[i]
        response_start, response_end = parsing(line)
        """
        초당 최대 처리량은
        요청의 응답 완료 여부에 관계없이
        임의 시간부터 1초(=1,000밀리초)간 처리하는 요청의 최대 개수를 의미
        """
        response_1s_end = response_end + 1000
        count = 1

        for j in range(i + 1, LEN):
            line = lines[j]
            next_response_start, next_response_end = parsing(line)

            # 서버에는 타임아웃이 3초로 적용되어 있기 때문에 처리시간은 0.001 ≦ T ≦ 3.000
            # For Testcase 3:
            # response_end + (1s : 초당 최대 처리량을 계산) + 3s with no response -> timeout
            if response_end + 1000 + 3000 <= next_response_start: # [respose_start, response_end]이므로 <=
                break

            if next_response_start < response_1s_end:
                count += 1

        answer = max(answer, count)
    return answer

In [ ]:
"""
Solution 2:
Two pointer
O(n)
- ! add 1000ms at the end of end_response
    to calculate collapsed time range easily
"""

def parsing(line):
    # time conversion to ms
    tc = {
        "s": 1000,
        "m": 1000 * 60,
        "h": 1000 * 60 * 60
    }

    # 로그 문자열마다 요청에 대한 응답완료시간 S와 처리시간 T가 공백으로 구분
    # S는 작년 추석인 2016년 9월 15일만 포함하여 고정 길이 2016-09-15 hh:mm:ss.sss 형식
    _, S, T = line.split()
    h, m, s = S.split(":")
    s, ms = s.split(".")

    # 처리시간 T는 최대 소수점 셋째 자리까지 기록하며 뒤에는 초 단위를 의미하는 s
    T = T.rstrip("s")
    T_s, T_ms = 0, 0
    if "." in T:
        T_s, T_ms = T.split(".")
    else:
        T_s = T

    # Convert time into ms
    h, m, s, ms, T_s, T_ms = tuple(map(int, [h, m, s, ms, T_s, T_ms]))
    end_response = ms + (s * tc["s"]) + (m * tc["m"]) + (h * tc["h"])

    # 처리시간은 시작시간과 끝시간을 포함
    T = T_ms + (T_s * tc["s"])
    start_response = end_response - T + 1

    # Add 1000ms for calculating 초당 최대 처리량
    return (start_response, end_response + 1000)


def solution(lines):
    answer = 0

    starts = list()
    ends = list()
    for line in lines:
        start_response, end_response = parsing(line)
        starts.append(start_response)
        ends.append(end_response)

    starts.sort()
    max_count, count = 0, 0
    s_idx, e_idx = 0, 0
    LEN = len(lines)
    while s_idx < LEN and e_idx < LEN:
        # s_idx log range is included in e_idx log range
        if starts[s_idx] < ends[e_idx]:
            s_idx += 1
            count += 1
            max_count = max(max_count, count)
        else:
            count -= 1 # remove count of e_idx log range
            e_idx += 1

    answer = max_count
    return answer